# Boilerplate

Packae installation, loading, and dataloaders. There's also a simple model defined. You can change it your favourite architecture if you want.

In [ ]:
!pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time

from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

# Implement the Attacks

Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.

In [ ]:
# The last argument 'targeted' can be used to toggle between a targeted and untargeted attack.
def fgsm(model, x, target, eps, targeted=True):
    ###############################################
    x.requires_grad_()
    L = nn.CrossEntropyLoss()
    loss = L(model(x), target)
    loss.backward()
    if targeted:
        adv_x = x - eps * torch.sign(x.grad)
    else:
        adv_x = x + eps * torch.sign(x.grad)
    adv_x = torch.clamp(adv_x, x - eps, x + eps)
    adv_x = torch.clamp(adv_x, 0, 1)
    ###############################################
    return adv_x


def pgd_untargeted(model, x, labels, k, eps, eps_step):
    ###############################################
    adv_x = x.clone().detach().requires_grad_()
    L = nn.CrossEntropyLoss()
    for i in range(k):
        adv_x_ = adv_x.clone().detach().requires_grad_()
        loss = L(model(adv_x_), labels)
        loss.backward()
        adv_x = adv_x + eps_step * torch.sign(adv_x_.grad)
        adv_x = torch.clamp(adv_x, x - eps, x + eps)
        adv_x = torch.clamp(adv_x, 0, 1)
    ###############################################
    return adv_x

# Implement Adversarial Training

In [ ]:
k = 40
eps = 0.1
eps_step = 0.01
num_epochs = 5

def train_model(model, num_epochs, enable_defense=True):
    learning_rate = 0.0001

    opt = optim.Adam(params=model.parameters(), lr=learning_rate)

    ce_loss = torch.nn.CrossEntropyLoss()

    tot_steps = 0

    for epoch in range(1,num_epochs+1):
        t1 = time.time()
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            if enable_defense:
                ###############################################
                model.eval()
                x_batch = pgd_untargeted(model, x_batch, y_batch, k, eps, eps_step)
                model.train()
                ###############################################

            tot_steps += 1
            opt.zero_grad()
            out = model(x_batch)
            batch_loss = ce_loss(out, y_batch)
            batch_loss.backward()
            opt.step()

        tot_test, tot_acc = 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_test += x_batch.size()[0]
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))

# Study Accuracy, Quality, etc.

Compare the various results and report your observations on the submission.

In [ ]:
print("Training original model:")
train_model(model, num_epochs, enable_defense=False)
model.eval()
# Test FGSM attack on original model
tot_test, tot_acc = 0.0, 0.0
t1 = time.time()
for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    x_batch = fgsm(model, x_batch, y_batch, eps, targeted=False)
    out = model(x_batch)
    pred = torch.max(out, dim=1)[1]
    acc = pred.eq(y_batch).sum().item()
    tot_acc += acc
    tot_test += x_batch.size()[0]
t2 = time.time()
print('FGSM on original model: Accuracy %.5lf [%.2lf seconds]' % (tot_acc/tot_test, t2-t1))
# Test PGD attack on original model
tot_test, tot_acc = 0.0, 0.0
t1 = time.time()
for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    x_batch = pgd_untargeted(model, x_batch, y_batch, k, eps, eps_step)
    out = model(x_batch)
    pred = torch.max(out, dim=1)[1]
    acc = pred.eq(y_batch).sum().item()
    tot_acc += acc
    tot_test += x_batch.size()[0]
t2 = time.time()
print('PGD on original model: Accuracy %.5lf [%.2lf seconds]' % (tot_acc/tot_test, t2-t1))
# PGD adversarial training
model.train()
print("PGD adversarial training:")
train_model(model, num_epochs, enable_defense=True)
model.eval()
# Test FGSM attack on enhanced model
tot_test, tot_acc = 0.0, 0.0
t1 = time.time()
for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    x_batch = fgsm(model, x_batch, y_batch, eps, targeted=False)
    out = model(x_batch)
    pred = torch.max(out, dim=1)[1]
    acc = pred.eq(y_batch).sum().item()
    tot_acc += acc
    tot_test += x_batch.size()[0]
t2 = time.time()
print('FGSM on enhanced model: Accuracy %.5lf [%.2lf seconds]' % (tot_acc/tot_test, t2-t1))
# Test PGD attack on enhanced model
tot_test, tot_acc = 0.0, 0.0
t1 = time.time()
for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    x_batch = pgd_untargeted(model, x_batch, y_batch, k, eps, eps_step)
    out = model(x_batch)
    pred = torch.max(out, dim=1)[1]
    acc = pred.eq(y_batch).sum().item()
    tot_acc += acc
    tot_test += x_batch.size()[0]
t2 = time.time()
print('PGD on enhanced model: Accuracy %.5lf [%.2lf seconds]' % (tot_acc/tot_test, t2-t1))